<a href="https://colab.research.google.com/github/VellummyilumVinoth/Aspect_based_sentimental_analysis/blob/main/daraz_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers
! pip install sentencepiece

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import nltk
import torch
import pandas as pd
import spacy
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification, BertTokenizer

nltk.download("punkt")

def add_full_stop(text):
    last_character = text[-1]
    if last_character != ".":
        text += "."
    return text

# Read the CSV file and extract the comments column
data = pd.read_csv("/content/drive/MyDrive/zidan/daraz_predictions.csv")
comments = data["Review"]
category = data["Category"]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
data

,Product Name,Category,Rating,Reviewer Name,Review,Review_without_emoji,preprocess_text,preprocess_text1,price,quality,delivery cost,packaging,design,customer service,usability
0,"Sunlight Care Detergent Liquid, 1L",Laundry,1.0,Eranga D,I purchased 2 bottles. Both seem partially use...,I purchased 2 bottles. Both seem partially use...,i purchased 2 bottles both seem partially used...,"['purchased', '2', 'bottles', 'seem', 'partial...",1,1,0,1,0,0,1
1,"Sunlight Care Detergent Liquid, 1L",Laundry,5.0,Suranga Y,Very fast & safe delivery . Neet packing. I wi...,Very fast & safe delivery . Neet packing. I wi...,very fast safe delivery neet packing i wish ...,"['fast', 'safe', 'delivery', 'neet', 'packing'...",1,0,1,1,0,0,0
2,"Sunlight Care Detergent Liquid, 1L",Laundry,5.0,Ishini F,Fast delivery during this pandemic. Got the pr...,Fast delivery during this pandemic. Got the pr...,fast delivery during this pandemic got the pro...,"['fast', 'delivery', 'pandemic', 'got', 'produ...",0,0,1,1,0,1,0
3,"Sunlight Care Detergent Liquid, 1L",Laundry,1.0,Dr. K,Alot of liquid has leaked.im very much disappo...,Alot of liquid has leaked.im very much disappo...,alot of liquid has leakedim very much disappoi...,"['alot', 'liquid', 'leakedim', 'much', 'disapp...",0,0,0,1,0,0,0
4,"Sunlight Care Detergent Liquid, 1L",Laundry,5.0,Rajitha S.,Thank you very much !! I received my package t...,Thank you very much !! I received my package t...,thank you very much i received my package tod...,"['thank', 'much', 'received', 'package', 'toda...",0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837,Sustagen Nutritional Supplement Chocolate Flav...,MilkPowder,4.0,by ******576,very good product value for the price,very good product value for the price,very good product value for the price,"['good', 'product', 'value', 'price']",1,0,0,0,0,0,0
1838,Sustagen Nutritional Supplement Chocolate Flav...,MilkPowder,4.0,by ruchira K.,Delivery within 1day to Tangalle... well packe...,Delivery within 1day to Tangalle... well packe...,delivery within 1day to tangalle well packedco...,"['delivery', 'within', '1day', 'tangalle', 'we...",1,0,1,1,0,1,0
1839,Sustagen Nutritional Supplement Chocolate Flav...,MilkPowder,3.0,by Sivathas J.,high quality reasonable price,high quality reasonable price,high quality reasonable price,"['high', 'quality', 'reasonable', 'price']",1,1,0,0,0,0,0
1840,Sustagen Nutritional Supplement Chocolate Flav...,MilkPowder,4.0,by Poornamith,Excellent packaging. And it was delivered quic...,Excellent packaging. And it was delivered quic...,excellent packaging and it was delivered quick...,"['excellent', 'packaging', 'delivered', 'quick...",1,1,1,1,0,1,0


In [31]:
# Initialize tokenizer and model for T5 text generation
tokenizer = T5Tokenizer.from_pretrained("flax-community/t5-base-wikisplit")
model = T5ForConditionalGeneration.from_pretrained("flax-community/t5-base-wikisplit")

# Initialize tokenizer and model for aspect sentiment analysis
aspect_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
aspect_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Initialize tokenizer and model for aspect classification
classification_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/zidan/finetuned_bert', num_labels=7)
classification_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/zidan/finetuned_bert', do_lower_case=True)

def assign_sentiment_label(sentiment_score):
    if sentiment_score >= 0.99983:
        sentiment_label = 'very good'
    elif sentiment_score >= 0.999:
        sentiment_label = 'good'
    elif sentiment_score > 0.001:
        sentiment_label = 'neutral'
    elif sentiment_score > 0.0002:
        sentiment_label = 'bad'
    else:
        sentiment_label = 'poor'
    return sentiment_label


# Define aspect labels
aspect_labels = ["price", "quality", "delivery cost", "packaging", "design", "customer service", "usability"]

# Initialize spaCy for sentence tokenization
nlp = spacy.load("en_core_web_sm")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Initialize the results list to store the processed data
results = []

# Process each comment and its sentences
for comment in comments:
    sentences = sent_tokenize(comment)

    for sentence in sentences:
        # Add full stop if necessary
        input_sentence = add_full_stop(sentence)

        # Tokenize and generate simple sentences using T5 model
        encoder_max_length = 256
        decoder_max_length = 256
        complex_tokenized = tokenizer(input_sentence, padding="max_length", truncation=True, max_length=encoder_max_length, return_tensors='pt')
        simple_tokenized = model.generate(complex_tokenized['input_ids'], attention_mask=complex_tokenized['attention_mask'], max_length=decoder_max_length, num_beams=5)
        simple_sentences = tokenizer.batch_decode(simple_tokenized, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        # Perform aspect classification
        input_statements = [statement.strip() for statement in simple_sentences if statement.strip()]

        encoded_inputs = classification_tokenizer.batch_encode_plus(
            input_statements,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512
        )

        with torch.no_grad():
            classification_model.eval()
            outputs = classification_model(**encoded_inputs)
            predicted_labels = outputs.logits

        predicted_labels = (torch.sigmoid(predicted_labels) > 0.5).to(torch.int).tolist()

        # Perform aspect sentiment analysis for each aspect
        aspect_sentiment_labels = {}
        aspect_sentiment_scores = {}

        for aspect_label in aspect_labels:
            aspect_sentiment_labels[aspect_label] = []
            aspect_sentiment_scores[aspect_label] = []

        for statement_index, statement in enumerate(input_statements):
            for aspect_index, aspect_label in enumerate(aspect_labels):
                if predicted_labels[statement_index][aspect_index] == 1:
                    inputs = aspect_tokenizer.encode_plus(statement, add_special_tokens=True, return_tensors="pt")
                    input_ids = inputs["input_ids"]
                    attention_mask = inputs["attention_mask"]

                    with torch.no_grad():
                        outputs = aspect_model(input_ids, attention_mask=attention_mask)
                        logits = outputs.logits

                    probabilities = torch.softmax(logits, dim=1)
                    sentiment_score = probabilities[0][1].item()  # Positive sentiment score
                    sentiment_label = assign_sentiment_label(sentiment_score)

                    aspect_sentiment_labels[aspect_label].append(sentiment_label)
                    aspect_sentiment_scores[aspect_label].append(sentiment_score)
                else:
                    aspect_sentiment_labels[aspect_label].append("-")
                    aspect_sentiment_scores[aspect_label].append("-")

        # Calculate the average sentiment score for each aspect
        overall_sentiment_scores = []
        for aspect_label in aspect_labels:
            numeric_scores = [score for score in aspect_sentiment_scores[aspect_label] if isinstance(score, float)]
            if numeric_scores:
                aspect_sentiment_scores[aspect_label] = sum(numeric_scores) / len(numeric_scores)
            else:
                aspect_sentiment_scores[aspect_label] = "-"

        # Calculate the overall sentiment label and score for the input sentence
        numeric_scores = [score for score in aspect_sentiment_scores.values() if isinstance(score, float)]
        if numeric_scores:
            overall_sentiment_score = sum(numeric_scores) / len(numeric_scores)
            overall_sentiment_label = assign_sentiment_label(overall_sentiment_score)
        else:
            overall_sentiment_score = "-"
            overall_sentiment_label = "-"

        # Store the result in a dictionary
        result = {
            "Comment": comment,
            "Category": category,
            "Sentence": input_sentence,
            "Overall Sentiment Score": overall_sentiment_score,
            "Overall Sentiment Label": overall_sentiment_label,
        }

        # Calculate the average sentiment label for each aspect
        average_sentiment_labels = {}
        for aspect_label in aspect_labels:
            sentiment_labels = [label for label in aspect_sentiment_labels[aspect_label] if label != "-"]
            if sentiment_labels:
                average_sentiment_labels[aspect_label] = max(set(sentiment_labels), key=sentiment_labels.count)
            else:
                average_sentiment_labels[aspect_label] = "-"

            # Add aspect sentiment label to the result dictionary
            result[aspect_label] = average_sentiment_labels[aspect_label]

        # Append the result to the results list
        results.append(result)

# Create DataFrame from the results list
result_df = pd.DataFrame(results)

In [ ]:
result_df

In [ ]:
1# Create an empty dictionary to store the results for each comment
comment_results = {}

# Iterate over the results
for result in results:
    comment = result['Comment']
    category = result['Category']

    # Check if the comment is already in the dictionary
    if comment in comment_results:
        # Update the overall sentiment score and label
        overall_sentiment_score = comment_results[comment]['Overall Sentiment Score']
        overall_sentiment_label = comment_results[comment]['Overall Sentiment Label']
        num_results = comment_results[comment]['Num Results']

        # Convert sentiment scores to float if necessary
        overall_sentiment_score = float(overall_sentiment_score)
        new_sentiment_score = float(result['Overall Sentiment Score']) if result['Overall Sentiment Score'] != '-' else 0.0

        # Calculate the running average for the sentiment score
        overall_sentiment_score = (overall_sentiment_score * num_results + new_sentiment_score) / (num_results + 1)

        # Update the overall sentiment label if necessary
        if result['Overall Sentiment Label'] != '-':
            overall_sentiment_label = result['Overall Sentiment Label']

        # Update the comment's result with the updated values
        comment_results[comment]['Overall Sentiment Score'] = overall_sentiment_score
        comment_results[comment]['Overall Sentiment Label'] = overall_sentiment_label
        comment_results[comment]['Num Results'] += 1

        # Update the aspect sentiment labels
        for aspect_label in aspect_labels:
            if result[aspect_label] != '-':
                comment_results[comment][aspect_label].append(result[aspect_label])

    else:
        # Create a new entry for the comment in the dictionary
        comment_results[comment] = {
            'Comment': comment,
            "Category": category,
            'Overall Sentiment Score': float(result['Overall Sentiment Score']) if result['Overall Sentiment Score'] != '-' else 0.0,
            'Overall Sentiment Label': result['Overall Sentiment Label'],
            'Num Results': 1,
        }

        # Initialize the aspect sentiment labels
        for aspect_label in aspect_labels:
            if result[aspect_label] != '-':
                comment_results[comment][aspect_label] = [result[aspect_label]]
            else:
                comment_results[comment][aspect_label] = []

# Create a list to store the final results
final_results = []

# Convert the comment results dictionary into a list of dictionaries
for comment_result in comment_results.values():
    # Calculate the average sentiment label for each aspect
    average_sentiment_labels = {}
    for aspect_label in aspect_labels:
        sentiment_labels = comment_result[aspect_label]
        if sentiment_labels:
            average_sentiment_labels[aspect_label] = max(set(sentiment_labels), key=sentiment_labels.count)
        else:
            average_sentiment_labels[aspect_label] = "-"

    # Create the final result dictionary for the comment
    final_result = {
        'Comment': comment_result['Comment'],
        "Category": comment_result['Category'],
        'Overall Sentiment Score': comment_result['Overall Sentiment Score'],
        'Overall Sentiment Label': comment_result['Overall Sentiment Label'],
    }

    # Add the aspect sentiment labels to the final result dictionary
    for aspect_label in aspect_labels:
        final_result[aspect_label] = average_sentiment_labels[aspect_label]

    # Append the final result to the list
    final_results.append(final_result)

# Create DataFrame from the final results list
result_df = pd.DataFrame(final_results, columns=["Comment","Category" ,"Overall Sentiment Score", "Overall Sentiment Label"] + aspect_labels)
result_df.to_csv('/content/drive/MyDrive/zidan/daraz_sentiment_analysis.csv', index=False)


In [ ]:
result_df